In [1]:
import os, sys, re, glob
from pymongo import MongoClient
from com.ibm.watson.datasource.clinicaltrials import ClinicalStudy
from com.ibm.watson.datasource.concepts import get_concepts
import numpy as np
import pandas as pd
from com.ibm.watson.datasource.wga_dev_db import get_wga_drug_df
import datetime
from collections import OrderedDict
from bson.codec_options import CodecOptions
from bson.son import SON

In [47]:
levels = [ ["NCT02407054", "NCT02407054", "NCT02962401"], ["Drug A", "Drug B", "Drug C"] ]
tuples = list(zip(*levels))
index = pd.MultiIndex.from_tuples(tuples, names=['NCT_ID', 'Drug'])
s = pd.DataFrame( ["AA", "BB", "CC"] , index=index, columns=["RAN"])
print tuples
print index
print s
s.to_excel("/Users/singhv/Desktop/new_drug.xlsx")

[('NCT02407054', 'Drug A'), ('NCT02407054', 'Drug B'), ('NCT02962401', 'Drug C')]
MultiIndex(levels=[[u'NCT02407054', u'NCT02962401'], [u'Drug A', u'Drug B', u'Drug C']],
           labels=[[0, 0, 1], [0, 1, 2]],
           names=[u'NCT_ID', u'Drug'])
                   RAN
NCT_ID      Drug      
NCT02407054 Drug A  AA
            Drug B  BB
NCT02962401 Drug C  CC


In [2]:
client = MongoClient()
db = client.WATSON
opts = CodecOptions(document_class=SON)
ctgov = db.CTGOV.with_options(codec_options=opts)
base_dir = os.path.join("/Users/singhv/Documents/Data/CTGOV/latest")

In [2]:
#cs = ClinicalStudy()
#cs.load_from_file(os.path.join(base_dir, "xmls", "update", "NCT02953782.xml"))
#print cs.get_trial_id()
relevant_drugs = pd.read_excel("/Users/singhv/Desktop/trial_stats/Drugs_to_query_for_trials_MODIFIED.xlsx", 
                            converters={'Therapy_ID': str})
print relevant_drugs [ relevant_drugs['Therapy_ID'] == '5168' ]

   Therapy_ID     Therapy in Quest priory list NCI_CODE  NCIM_CUI  UMLS_CUI
27       5168  Danusertib                  NaN   C61587  C2700603  C2700603


In [3]:
print relevant_drugs [ relevant_drugs['UMLS_CUI'] == 'C0393022' ].any()['UMLS_CUI']

False


In [ ]:
new_drugs_dict = dict()
df_indices = []
umls_concepts = []
relevant_drug_exists = []
for count, xml in enumerate(glob.glob1(os.path.join(base_dir, "xmls", "update.old"), "*.xml")):
    if (count % 10 == 0):
        print "Processed %d (%s)" % (count, xml)
    cs = ClinicalStudy()
    try:
        cs.load_from_file(os.path.join(base_dir,"xmls", "update.old", xml))
        nct_id = cs.get_trial_id()
        interventions = cs.get_interventions()
        drug_names = interventions.get("Drug", [])
        db_trial = db.CTGOV.find_one({"NCT_ID" : nct_id})
        existing_interventions = db_trial["interventions"]
        existing_drug_names = [ x["NAME"] for x in existing_interventions ]
        non_existing_drugs = set(drug_names) - set(existing_drug_names)
        df_indices.extend( [(nct_id, drug_name) for drug_name in non_existing_drugs] )
        for drug_name in non_existing_drugs:
            concept = get_concepts(drug_name)
            if concept:
                umls_cui = concept[0]["UMLS_CUI"]
                umls_concepts.append(umls_cui)
                relevant_drug_exists.append("Y" if relevant_drugs[ relevant_drugs["UMLS_CUI"] == umls_cui ].any()['UMLS_CUI'] else "N" )
            else:
                umls_concepts.append("NONE")
                relevant_drug_exists.append("N")
    except:
        print "Exception processing: %s" % xml
            

print df_indices
index = pd.MultiIndex.from_tuples(df_indices, names=['NCT_ID', 'Drug'])
df = pd.DataFrame({"UMLS_CUI" : umls_concepts, "EXISTS" : relevant_drug_exists}, index=index)
print df

In [ ]:
print df.loc[(u'NCT00047060', u'Campath-1H')]
nct_cache = {}
for (nct_id, drug) in df.index.values:
    status = nct_cache.get(nct_id, db.CTGOV.find_one({"NCT_ID" : nct_id})["metadata"]["STATUS"])
    nct_cache[nct_id] = status
    df["STATUS"][(nct_id, drug)] = status
    
print df.head(10)

In [52]:
df.to_excel("/Users/singhv/Desktop/trial_stats/all_inscope_drug_trials.xlsx")

In [4]:
df = pd.read_excel("/Users/singhv/Desktop/trial_stats/all_inscope_drug_trials.xlsx", index_col=[0,1])

In [5]:
# Reshaped dataframe with trial id in every row
new_index_column = list()
for (trial_id, drug_name) in df.index.values:
    new_index_column.append(trial_id)
    
df["NEW_NCT_ID"] = new_index_column
df.to_excel("/Users/singhv/Desktop/trial_stats/all_inscope_drug_trials_reshaped.xlsx")

In [5]:
#indices = df[ (df["DRUG OF INTEREST"] == "Y") & (df["STATUS"] == "ACCEPTED") ].index.values
indices = df.index.values
trial_ids = set([ index[0] for index in indices ])
print len(trial_ids)
#print df.loc[(u'NCT01042379', )]

2744


In [24]:
#print df.loc[(u'NCT01042379', )].shape
#for row in df.loc[(u'NCT01042379', )].itertuples():
#    print row[2]
inscope_trial_ids = set( [x[0] for x in df [ df["DRUG OF INTEREST"] == "Y" ].index] )
#print df [ df["DRUG OF INTEREST"] == "Y" ].index[0]
print len(inscope_trial_ids)
print db.CTGOV.find({"NCT_ID" : {"$in" : list(inscope_trial_ids)}, "metadata.STATUS" : "PENDING"}).count()
#update = db.CTGOV.update_many({"NCT_ID" : {"$in" : list(inscope_trial_ids)}, "metadata.STATUS" : "PENDING"}, {"$set" : {"metadata.STATUS" : "ACCEPTED"}})

#print update.modified_count

535
0


In [24]:
for i, trial_id in enumerate(trial_ids):
    if i % 500 == 0: 
        print "Processed %d" % i
    study = db.CTGOV.find_one({"NCT_ID" : trial_id})
    existing_drug_names = set([ x["NAME"] for x in study["interventions"] ])
    new_status = "PENDING" if (study["metadata"]["STATUS"] == "ACCEPTED") else study["metadata"]["STATUS"]
    interventions = list()
    for row in df.loc[(trial_id, )].itertuples():    
        if not (row[0] in existing_drug_names):
            intervention = [ ("ID", row[2]), ("NAME", row[0]), ("PREFERRED_NAME", get_preferred_term(row[2])),
                           ("SOURCE", "ctm"), ("TYPE", "Drug"), ("EDIT", False) ]
            interventions.append(OrderedDict(intervention))
    db.CTGOV.update_one({"NCT_ID" : trial_id}, 
                        {"$push" : {"interventions" : {"$each" : interventions}},
                         "$set" : {"metadata.STATUS" : new_status, "metadata.UPDATED_BY" : "SYSTEM",
                                  "metadata.LAST_UPDATED" : datetime.datetime.utcnow()}})

Processed 0
Processed 500
Processed 1000
Processed 1500
Processed 2000
Processed 2500
Processed 3000
Processed 3500
Processed 4000
Processed 4500
Processed 5000


In [5]:
wga_drug_df = get_wga_drug_df()
result = wga_drug_df[ wga_drug_df['UMLS_CUI'] == "C0210630" ]
print result["THERAPY_ID"]

Series([], Name: THERAPY_ID, dtype: object)


In [27]:
# Updating trials with CL CUIs replace with UMLS CUIs
#wga_drug_df = get_wga_drug_df()
#print wga_drug_df.head(2)
count = 0
for study in db.CTGOV.find({"$query": {}, "$snapshot" : True}):
    for intervention in study["interventions"]:
        cui = intervention.get("ID")
        umls_cui = None
        wgadb_id = intervention.get("WGADB_ID")
        if (cui is not None) and (cui != ""):
            if "CL" in cui:
                nci_record = db.NCIT.find_one({"NCIM_CUI" : cui})
                if nci_record and nci_record.get("UMLS_CUI"):
                    intervention["ID"] = nci_record.get("UMLS_CUI")
                    umls_cui = nci_record.get("UMLS_CUI")
            else:
                umls_cui = cui
            if (wgadb_id is None) and (umls_cui is not None):
                result = wga_drug_df[ wga_drug_df["UMLS_CUI"] == umls_cui ]
                if result["THERAPY_ID"].any():
                    intervention["WGADB_ID"] = result["THERAPY_ID"].any()
                    
    db.CTGOV.save(study)
    #count += db.CTGOV.save(study)["nModified"]
    #print "Current count: %d" % count
    
#print "Count: %d" % count

In [4]:
def merge_interventions(int1, int2):
    curated = int1 if "EDIT" in int2 else int2
    latest = int1 if "EDIT" in int1 else int2
    curated["NAME"] = latest["NAME"]
    curated["EDIT"] = False
    return curated

In [31]:
# UPdating trials to merge interventions which only differ due to case
unchanged_trials = list()
changed_trials = list()
count = 0
for study in db.CTGOV.find({"$query": {"metadata.STATUS":"PENDING"}, "$snapshot" : True}):
#for study in ctgov.find({"$query": {"NCT_ID":"NCT01951885"}, "$snapshot" : True}):
    curated_interventions = [ x for x in study["interventions"] if "EDIT" not in x ]
    interventions = sorted( study["interventions"], key=lambda x: x["NAME"].lower() )
    #print "\n".join([ x["NAME"] for x in interventions ])
    new_intervention_array = []
    i = 0
    while ( i < len(interventions) ):
        if i < (len(interventions) - 1):
            if interventions[i]["NAME"].lower() == interventions[i + 1]["NAME"].lower():
                new_intervention_array.append(merge_interventions(interventions[i], interventions[i + 1]))
                i += 2
            else:
                new_intervention_array.append(interventions[i])
                i += 1
        else:
            new_intervention_array.append(interventions[i])
            i += 1
    for intervention in new_intervention_array:
        if "ID" in intervention and (intervention.get("PREFERRED_NAME") == ""):
            record = db.NCIT.find_one({"UMLS_CUI" : intervention["ID"]})
            if record:
                intervention["PREFERRED_NAME"] = record["PREF_NAME"]
    if len(curated_interventions) == len(new_intervention_array):
        count += ctgov.update_one({"NCT_ID":study["NCT_ID"]},{"$set" : {"interventions" : new_intervention_array, "metadata.STATUS" : "ACCEPTED"}}).matched_count
        print "Same length: %s" % study["NCT_ID"]
    else:
        count += ctgov.update_one({"NCT_ID":study["NCT_ID"]},{"$set" : {"interventions" : new_intervention_array}}).matched_count
    
print
#print "\n".join([ x["NAME"] for x in new_intervention_array ])
#print len(new_intervention_array)
print "Updated: %d" % count


Updated: 1376


In [34]:
#Updating all trials to add EDIT in interventions
count = 0
for study in ctgov.find({"$query": {}, "$snapshot" : True}):
    new_intervention_array = []
    for intervention in study["interventions"]:
        if "EDIT" not in intervention:
            intervention["EDIT"] = True
        new_intervention_array.append(intervention)
        
    count += ctgov.update_one({"NCT_ID":study["NCT_ID"]},{"$set" : {"interventions" : new_intervention_array}}).matched_count
    
print "Total updated: %d" % count

Total updated: 6749


In [25]:
# Setting all PENDING trials with in-scope drugs to ACCEPTED
pending_trials = []
for trial in db.CTGOV.find({"metadata.STATUS" : "PENDING"}):
    drug_umls_cuis = [ therapy["ID"] for therapy in trial["interventions"] if "ID" in therapy ]
    if relevant_drugs['UMLS_CUI'].isin(drug_umls_cuis).any():
        pending_trials.append(trial["NCT_ID"])
        
print len(pending_trials)

267


In [6]:
client_prod = MongoClient("bioresearch-d1:23001")
db_prod = client_prod["WATSON"]
trials_with_inscope_drugs = []
for trial in db_prod.CTGOV.find({"COUNTRIES":"United States"}):
    drug_umls_cuis = [ therapy["ID"] for therapy in trial["interventions"] if "ID" in therapy ]
    if relevant_drugs['UMLS_CUI'].isin(drug_umls_cuis).any():
        trials_with_inscope_drugs.append(trial["NCT_ID"])
        
print len(trials_with_inscope_drugs)

1458


In [12]:
print len(trials_with_inscope_drugs)
print trials_with_inscope_drugs[:10]
temp = db_prod["CTGOV"].find({"NCT_ID" : {"$in" : trials_with_inscope_drugs},"metadata.DOWNLOAD_DATE" : {"$gt" : datetime.datetime(2017,3,27,0,0)}})
print temp.count()

1458
[u'NCT02206763', u'NCT02610075', u'NCT02576444', u'NCT02413853', u'NCT02457793', u'NCT01548144', u'NCT02389309', u'NCT01884740', u'NCT01532011', u'NCT02155920']
14


In [7]:
print db_prod.CTGOV.update_many({"NCT_ID" : {"$in" : trials_with_inscope_drugs}}, {"$set":{"metadata.REVIEWER":"pmcneil"}}).modified_count

14


In [25]:
result = wga_drug_df["THERAPY_ID"][ wga_drug_df['UMLS_CUI'] == "C3661275" ]
if result.any():
    print "ha"

In [ ]:
count = 0
count1 = 0
for study in db.CTGOV.find({"interventions.WGADB_ID" : {"$ne" : True}}):
    #temp = [ x for x in study["interventions"] if "WGADB_ID" not in x ]
    #count += len(temp)
    print study["NCT_ID"]
    for intervention in study["interventions"]:
        if ("WGADB_ID" not in intervention):
            count += 1
            print "\t" + intervention.get("ID", "") 
            result = wga_drug_df["THERAPY_ID"][ wga_drug_df['UMLS_CUI'] == intervention.get("ID") ]
            if result.any(): count1 += 1
    
print "Count is: %d" % count
print "Count1 is: %d" % count1

In [3]:
no_interventions = [ x["NCT_ID"] for x in db.CTGOV.find({"metadata.STATUS" : "ACCEPTED", "interventions" : {"$size" : 0}}) ]
no_conditions = [ x["NCT_ID"] for x in db.CTGOV.find({"metadata.STATUS" : "ACCEPTED", "conditions" : {"$size" : 0}}) ]

interventions_no_id = [ x["NCT_ID"] for x in db.CTGOV.find({"metadata.STATUS" : "ACCEPTED", "interventions.ID" : {"$exists" : False}}) ] 
conditions_no_id = [ x["NCT_ID"] for x in db.CTGOV.find({"metadata.STATUS" : "ACCEPTED", "conditions.ID" : {"$exists" : False}}) ]

no_interventions = set(no_interventions)
no_conditions = set(no_conditions)
interventions_no_id = set(interventions_no_id)
conditions_no_id = set(conditions_no_id)

In [4]:
print "Length no interventions: %d" % len(no_interventions)
print "Length no conditions: %d" % len(no_conditions)
print "Length interventions no id: %d" % len(interventions_no_id)
print "Length conditions no id: %d" % len(conditions_no_id)
print no_interventions.issubset(no_conditions)
print interventions_no_id.issubset(no_interventions)
print conditions_no_id.issubset(no_conditions)
print len(no_conditions.union(no_interventions))

Length no interventions: 14
Length no conditions: 69
Length interventions no id: 14
Length conditions no id: 69
False
True
True
71


In [76]:
genes = "erbb2|her2|alk|ros|ros1|ret|braf|mek|akt1|pi3k|pi3kca|mtor|abl1|abl|pdgfr|pdgfra|pdgfrb|kit|myc|met|jak"
genes = genes + "|egfr|fgfr|vegfr"
genes = genes + "|" + genes.upper()
expression = "pos|neg|positive|negative|expressed|expression|\+|\-"
#genes = "CML|AML"
#print genes
genes_regex = re.compile("(\w+\s\w+\s|\w+\W+)(%s)(\s*\w+\s*\w+|\w+\W+)" % (genes))
expr_regex = re.compile(expression, flags=re.IGNORECASE)
nct_ids, phrases = ([], [])
for study in db.CTGOV.find({}):
    inclusion = study["inclusion"].get("text", "")
    match = genes_regex.findall(inclusion)
    if match:
        for m in match:
            phrase = m[0] + m[1] + m[2]
            if expr_regex.search(phrase):
                print study["NCT_ID"] + ": " + phrase
                nct_ids.append(study["NCT_ID"])
                phrases.append(phrase)
                
output_df = pd.DataFrame(index=nct_ids)              
#output_df["NCT_ID"] = nct_ids
output_df["PHRASE"] = phrases
output_df.to_excel("/Users/singhv/Desktop/gene_expression_phrases.xlsx")

NCT01956669: 131 -
             meta-
NCT01738139: c-KIT mutations by
NCT02412722: postovulation methods
NCT02412722: postovulation methods for
NCT01975701: translocation to FGFR1-
NCT01975701: TACC1, FGFR3-
NCT01787500: with positive BRAF V600 mutation
NCT02091141: abnormalities:

          -  HER2 overexpression
NCT02091141: that have HER2 overexpression
NCT02091141: Vemurafenib

          -  BRAF mutation positivity
NCT02438722: non-metastatic disease
NCT01723202: post ovulation methods
NCT01519323: Positive BRAF mutation result
NCT01776008: and HER2 negative
NCT01787487: BCR-ABL1 negative
NCT02448771: required.

          -  HER2 negative
NCT02448771: negative (HER2 copy
             number
NCT01933815: cis-retinoic
                  acid
NCT02047214: cis-retinoic
                  acid
NCT02143414: BCR-ABL status must
NCT01352520: CD30+ mycosis fungoides
NCT01441947: and HER2 negative breast
NCT01868022: FGF2 or FGFR1 expression
NCT01792050: HER2)-, metastatic breast
NCT02457910: 

In [3]:
existing_pending_trials = [ x["NCT_ID"] for x in db.CTGOV.find({"metadata.STATUS":"PENDING"}) ]
print len(existing_pending_trials)

1729


In [6]:
previous_pending_trials = [ x["NCT_ID"] for x in db.CTGOV.find({"metadata.STATUS":"PENDING"}) ]
print len(previous_pending_trials)

738


In [8]:
print set(existing_pending_trials).issuperset(set(previous_pending_trials))

True


In [9]:
difference = set(existing_pending_trials).difference(set(previous_pending_trials))
print len(difference)

991


In [37]:
print len(difference)
difference_status = [ (x["NCT_ID"], x["metadata"]["STATUS"]) for x in db.CTGOV.find({"NCT_ID" : {"$in" : list(difference)}}) ]
print len(difference_status)

991
991


In [22]:
client_prod = MongoClient("bioresearch-d1:23001")
db_prod = client_prod.WATSON


In [26]:
print db_prod.CTGOV.find({"metadata.STATUS":"PENDING"}).count()

1729


In [40]:
difference_status = dict(difference_status)
count_new = 0
count_working = 0
for (nct_id, status) in difference_status.iteritems():
    if status == "NEW": count_new += 1
    if status == "WORKING": count_working += 1
        
print count_new
print count_working

302
11


In [42]:
count = 0
for (nct_id, status) in difference_status.iteritems():
    if status in ["NEW", "WORKING"]:
        count += db_prod.CTGOV.update_one({"NCT_ID" : nct_id}, {"$set" : {"metadata.STATUS" : status}}).modified_count
    
print count

313


In [2]:
cs = ClinicalStudy("NCT00924170")
cs.get()
print cs.get_title()
print cs.get_gender()

Phase II Trial of LMB-2, Fludarabine and Cyclophosphamide for Adult T-Cell Leukemia
All


In [2]:
client = MongoClient("bioresearch-d1:23001")
db = client["WATSON"]

In [3]:
studies = db["CTGOV"].find({"metadata.DOWNLOAD_DATE" : {"$gt" : datetime.datetime(2017,3,27,0,0)}})
#studies = db["CTGOV"].find({"GENDER" : {"$exists" : False}})
print studies.count()
nct_ids = [ study["NCT_ID"] for study in studies ]

130


In [8]:
study = db["CTGOV"].find_one({"NCT_ID" : 'TEST'})
print study["inclusion"]["GROUP_OPERATORS"] == []

for nct_id in nct_ids:
    print nct_id
    cs = ClinicalStudy(nct_id)
    cs.get()
    criteria = cs.get_raw_criteria()
    study = db["CTGOV"].find_one({"NCT_ID" : nct_id})
    inclusion = study["inclusion"]
    exclusion = study["exclusion"]
    
    inclusion["text"] = criteria["inclusion"]
    if inclusion.get("GROUP_OPERATORS") == []:
        inclusion["GROUP_OPERATORS"] = ["NONE"]
        
    if inclusion.get("variantGroups") == []:
        inclusion["variantGroups"] = [ {"OPERATOR" : "OR", "ID" : 1, "VARIANT_IDS" : []} ]
        
    exclusion["text"] = criteria["exclusion"]
    if exclusion.get("GROUP_OPERATORS") == []:
        exclusion["GROUP_OPERATORS"] = ["NONE"]
        
    if exclusion.get("variantGroups") == []:
        exclusion["variantGroups"] = [ {"OPERATOR" : "OR", "ID" : 1, "VARIANT_IDS" : []} ]
        
    db["CTGOV"].update_one({"NCT_ID" : nct_id}, {"$set" : {"inclusion" : inclusion, "exclusion" : exclusion}})


False
NCT02581059
NCT02078609
NCT03075696
NCT03073785
NCT03076372
NCT03076554
NCT03079427
NCT03079440
NCT02955823
NCT03072160
NCT03072771
NCT03072992
NCT03072134
NCT03075462
NCT03076437
NCT03074318
NCT03075553
NCT03075826
NCT03077659
NCT03077685
NCT03078855
NCT03079011
NCT01444807
NCT02063724
NCT02259621
NCT02302833
NCT02564536
NCT02765243
NCT03049358
NCT03071757
NCT03071874
NCT03072238
NCT03073525
NCT03074006
NCT03074825
NCT03075423
NCT03075527
NCT03077451
NCT03077698
NCT03078400
NCT03088930
NCT01806675
NCT02372409
NCT02681796
NCT02706392
NCT03000179
NCT03050047
NCT03071926
NCT03072043
NCT03076164
NCT03077243
NCT03078751
NCT03079505
NCT03079999
NCT03080116
NCT03080311
NCT03080805
NCT03080922
NCT03080974
NCT03081039
NCT03081143
NCT03081234
NCT03081377
NCT03081481
NCT03081494
NCT03081689
NCT03081702
NCT03081715
NCT03081780
NCT03081858
NCT03081910
NCT03081923
NCT03082144
NCT03082209
NCT03082534
NCT03083041
NCT03083054
NCT03083613
NCT03083678
NCT03083691
NCT03083808
NCT03083873
NCT0308423

In [16]:
with open("/Users/singhv/Desktop/genders.txt", "wb") as f:
    for nct_id in nct_ids:
        cs = ClinicalStudy(nct_id)
        cs.get()
        print cs.get_gender()
        db["CTGOV"].update_one({"NCT_ID" : nct_id}, {"$set" : {"GENDER" : cs.get_gender()}})
        f.write(nct_id + "\t" + cs.get_gender() + "\n")

All
All
All
All
All
All
All
All
All
All
All
Female
All
Female
All
All
All
All
Male
All
All
Female
All
All
Female
All
All
All
All
All
All
All
Male
Female
All
All
All
All
All
All
Female
All
Female
All
All
All
All
All
All
Female
All
All
All
All
All
All
Male
All
All
All
All
All
All
All
All
Male
All
All
Female
All
All
All
All
Male
All
All
All
All
All
All
All
All
All
All
Female
All
All
All
All
All
All
All
Female
All
All
All
Male
All
All
All
All
All
All
Female
Female
All
All
All
All
All
All
All
All
Male
All
Male
All
Female
All
All
All
All
All
Male
All
All
All
All
All
All
